In [89]:
import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [90]:
# Load data of IT closing transaction
df = pd.read_csv('../CSV/ClosingTransaction_TPIT.csv', sep=';')

df.head()


,GjaeRecId,AccountingDate,CreatedDateTime,LedgerDimension,GroupChartOfAccountsValue,GroupChartOfAccountsName,Ledger,PostingLayer,SubledgerVoucher,SubledgerVoucherDataAreaId,...,MainAccount,Text,JournalType,JournalTypeLabel,NumOfLines,CountLedgerJournal,TransactionCurrencyAmount,TransactionCurrencyCode,ClosingActivityDescription,ClosingActivityId
0,5637225788,31/12/2017 00:00,11/12/2017 16:37,5637173886,E41060IC,Consultant Costs,5637145326,0,PR000001,IT01,...,5637145693,"DDT Nr: Ceccomori-cons.12/17, Internal Nr: PR0...",NaN,NaN,NaN,NaN,1733680000,EUR,Post vendor invoice,ACT01
1,5637225789,31/12/2017 00:00,11/12/2017 16:37,5637173897,L20201,Trade Accounts Payable - Bill Not Receiv,5637145326,0,PR000001,IT01,...,5637145529,"DDT Nr: Ceccomori-cons.12/17, Internal Nr: PR0...",NaN,NaN,NaN,NaN,-1733680000,EUR,Post vendor invoice,ACT01
2,5637233421,12/01/2017 00:00,11/12/2017 20:14,5637175112,E41040IC,Employee Benefits,5637145326,0,1100000037,it01,...,5637145765,LEASYS - M.C. Doria FE140WD 01/17,0.0,Daily,12149.0,20.0,326230000,EUR,Post all adjustments in ledger,ACT32
3,5637235692,24/01/2017 00:00,11/12/2017 20:14,5637175123,L20220,VAT Payable,5637145326,0,200000045,it01,...,5637145547,NaN,0.0,Daily,12149.0,4.0,-3883230000,EUR,Post all adjustments in ledger,ACT32
4,5637235730,30/01/2017 00:00,11/12/2017 20:14,5637176287,A10100,Trade Accounts Receivable,5637145326,0,200000059,it01,...,5637145443,ENEL SE-50046-12/16,0.0,Daily,12149.0,3.0,46090210000,EUR,Post and settle all received payments,ACT09


In [91]:
# Dataset dimension
df.shape

(835134, 24)

In [92]:
# Dataset info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 835134 entries, 0 to 835133
Data columns (total 24 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   GjaeRecId                   835134 non-null  int64  
 1   AccountingDate              835134 non-null  object 
 2   CreatedDateTime             835134 non-null  object 
 3   LedgerDimension             835134 non-null  int64  
 4   GroupChartOfAccountsValue   835134 non-null  object 
 5   GroupChartOfAccountsName    835134 non-null  object 
 6   Ledger                      835134 non-null  int64  
 7   PostingLayer                835134 non-null  int64  
 8   SubledgerVoucher            835134 non-null  object 
 9   SubledgerVoucherDataAreaId  835134 non-null  object 
 10  CreatedBy                   835134 non-null  object 
 11  JournalCategory             698624 non-null  object 
 12  PostingTypeLabel            826696 non-null  object 
 13  PostingTypeVal

In [93]:
# Change SubledgerVoucherDataAreaId field from lower case to upper case
df['SubledgerVoucherDataAreaId'] = df['SubledgerVoucherDataAreaId'].str.upper()

In [94]:
# Date format of CreatedDateTime column
df['CreatedDateTime'] = pd.to_datetime(df['CreatedDateTime'], format='%d/%m/%Y %H:%M')
df.head()

,GjaeRecId,AccountingDate,CreatedDateTime,LedgerDimension,GroupChartOfAccountsValue,GroupChartOfAccountsName,Ledger,PostingLayer,SubledgerVoucher,SubledgerVoucherDataAreaId,...,MainAccount,Text,JournalType,JournalTypeLabel,NumOfLines,CountLedgerJournal,TransactionCurrencyAmount,TransactionCurrencyCode,ClosingActivityDescription,ClosingActivityId
0,5637225788,31/12/2017 00:00,2017-12-11 16:37:00,5637173886,E41060IC,Consultant Costs,5637145326,0,PR000001,IT01,...,5637145693,"DDT Nr: Ceccomori-cons.12/17, Internal Nr: PR0...",NaN,NaN,NaN,NaN,1733680000,EUR,Post vendor invoice,ACT01
1,5637225789,31/12/2017 00:00,2017-12-11 16:37:00,5637173897,L20201,Trade Accounts Payable - Bill Not Receiv,5637145326,0,PR000001,IT01,...,5637145529,"DDT Nr: Ceccomori-cons.12/17, Internal Nr: PR0...",NaN,NaN,NaN,NaN,-1733680000,EUR,Post vendor invoice,ACT01
2,5637233421,12/01/2017 00:00,2017-12-11 20:14:00,5637175112,E41040IC,Employee Benefits,5637145326,0,1100000037,IT01,...,5637145765,LEASYS - M.C. Doria FE140WD 01/17,0.0,Daily,12149.0,20.0,326230000,EUR,Post all adjustments in ledger,ACT32
3,5637235692,24/01/2017 00:00,2017-12-11 20:14:00,5637175123,L20220,VAT Payable,5637145326,0,200000045,IT01,...,5637145547,NaN,0.0,Daily,12149.0,4.0,-3883230000,EUR,Post all adjustments in ledger,ACT32
4,5637235730,30/01/2017 00:00,2017-12-11 20:14:00,5637176287,A10100,Trade Accounts Receivable,5637145326,0,200000059,IT01,...,5637145443,ENEL SE-50046-12/16,0.0,Daily,12149.0,3.0,46090210000,EUR,Post and settle all received payments,ACT09


In [95]:
# Set all time of createdDateTime column to 00:00:00 to avoid error
df['CreatedDateTime'] = df['CreatedDateTime'].dt.strftime('%d/%m/%Y 00:00:00')

# Convert CreatedDateTime column to datetime format
df['CreatedDateTime'] = pd.to_datetime(df['CreatedDateTime'], format='%d/%m/%Y %H:%M:%S')
df.head()


,GjaeRecId,AccountingDate,CreatedDateTime,LedgerDimension,GroupChartOfAccountsValue,GroupChartOfAccountsName,Ledger,PostingLayer,SubledgerVoucher,SubledgerVoucherDataAreaId,...,MainAccount,Text,JournalType,JournalTypeLabel,NumOfLines,CountLedgerJournal,TransactionCurrencyAmount,TransactionCurrencyCode,ClosingActivityDescription,ClosingActivityId
0,5637225788,31/12/2017 00:00,2017-12-11,5637173886,E41060IC,Consultant Costs,5637145326,0,PR000001,IT01,...,5637145693,"DDT Nr: Ceccomori-cons.12/17, Internal Nr: PR0...",NaN,NaN,NaN,NaN,1733680000,EUR,Post vendor invoice,ACT01
1,5637225789,31/12/2017 00:00,2017-12-11,5637173897,L20201,Trade Accounts Payable - Bill Not Receiv,5637145326,0,PR000001,IT01,...,5637145529,"DDT Nr: Ceccomori-cons.12/17, Internal Nr: PR0...",NaN,NaN,NaN,NaN,-1733680000,EUR,Post vendor invoice,ACT01
2,5637233421,12/01/2017 00:00,2017-12-11,5637175112,E41040IC,Employee Benefits,5637145326,0,1100000037,IT01,...,5637145765,LEASYS - M.C. Doria FE140WD 01/17,0.0,Daily,12149.0,20.0,326230000,EUR,Post all adjustments in ledger,ACT32
3,5637235692,24/01/2017 00:00,2017-12-11,5637175123,L20220,VAT Payable,5637145326,0,200000045,IT01,...,5637145547,NaN,0.0,Daily,12149.0,4.0,-3883230000,EUR,Post all adjustments in ledger,ACT32
4,5637235730,30/01/2017 00:00,2017-12-11,5637176287,A10100,Trade Accounts Receivable,5637145326,0,200000059,IT01,...,5637145443,ENEL SE-50046-12/16,0.0,Daily,12149.0,3.0,46090210000,EUR,Post and settle all received payments,ACT09


In [96]:
# Create a new df with only CreatedDateTime and count of rows (transactions) per day and the field SubledgerVoucherDataAreaId
df_TS = df.groupby(['SubledgerVoucherDataAreaId', 'CreatedDateTime']).size().reset_index(name='count')
df_TS.head()

,SubledgerVoucherDataAreaId,CreatedDateTime,count
0,IT01,2017-12-11,13590
1,IT01,2017-12-12,66944
2,IT01,2017-12-13,83
3,IT01,2017-12-14,118
4,IT01,2017-12-15,8765


In [99]:
# Change name columns SubledgerVoucherDataAreaId to AreaId, CreatedDateTime to Timestamp and count to Target
df_TS = df_TS.rename(columns={'SubledgerVoucherDataAreaId': 'AreaId', 'CreatedDateTime': 'Timestamp', 'count': 'Target'})
df_TS.head()


,AreaId,Timestamp,Target
0,IT01,2017-12-11,13590
1,IT01,2017-12-12,66944
2,IT01,2017-12-13,83
3,IT01,2017-12-14,118
4,IT01,2017-12-15,8765


In [101]:
# Sum all values of Target in total
df_TS['Target'].sum()


835134